In [98]:
from bs4 import BeautifulSoup
import requests
import os
import csv

def parse_carrera_links(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Obtener enlaces de las carreras de diferentes categorías
        carrera_links_grado = [a['href'] for a in soup.select('div.grado a')]
        carrera_links_posgrado = [a['href'] for a in soup.select('div.posgrado a')]
        carrera_links_tecnologias = [a['href'] for a in soup.select('div.tecnologias a')]
        
        # Unir los enlaces de todas las categorías
        carrera_links = carrera_links_grado + carrera_links_posgrado + carrera_links_tecnologias
        
        return carrera_links
    else:
        print("No se pudo acceder a la página web.")
        return []


In [16]:

def scrape_and_save_data_grado(soup, writer):
   
    university_sections = soup.find_all('section', class_='wrapper-semestre')
    for section in university_sections:
        # Extrae el ciclo desde el div con la clase 'btn-semestre'
        ciclo = section.find('div', class_='btn-semestre').text.strip()
        career_title_div = soup.find('div', id='career-title')
        if career_title_div:
                # Extrae el nombre de la carrera desde el elemento <h2>
            carrera = career_title_div.find('h2').text.strip()
            facultad = career_title_div.find('p').text.strip()
        else:
            carrera = "Carrera no encontrada"
            facultad = "Facultad no encontrada"
    
        # Extrae el nombre de la universidad desde el elemento <p>
        universidad_elem = soup.find('p')
        if universidad_elem:
            universidad = universidad_elem.text.strip()
        else:
            universidad = "Universidad no encontrada"
    
        # Extrae los nombres de las materias desde los elementos <li> dentro del div con la clase 'link1 whitebg'
        materias_div = section.find('div', class_='link1 whitebg')
        materias = [li.text.strip() for li in materias_div.find_all('li')]
        # Transforma las modalidades según tus requisitos
        # Extrae las modalidades de inscripción
        modalidades_elem = soup.find('div', class_='boton-inscripcion').find_all('a')
        modalidades = [modalidad_elem.text.strip() for modalidad_elem in modalidades_elem]
        # Convertir todas las modalidades a minúsculas antes de realizar los reemplazos
        modalidades = [modalidad.lower() for modalidad in modalidades]
            
        # Realizar el reemplazo de la modalidad "presencial"
        modalidades = [modalidad.replace("modalidad presencial - inscríbete a la prueba de admisión", "presencial") for modalidad in modalidades]
            
        # Realizar el reemplazo de la modalidad "distancia"
        modalidades = [modalidad.replace("modalidad a distancia - matricúlate ahora", "distancia") for modalidad in modalidades]
    
    
        # Extraer el título que otorga
        titulo = soup.select(".item:nth-child(2) div")[0].get_text(strip=True)
            
                
        # Extraer la duración
        duracion = soup.select(".item:nth-child(3) div")[0].get_text(strip=True)
    
        # Extraer la descripción de la carrera
        descripcion_div = soup.find('div', class_='section1')
        descripcion_items = descripcion_div.find_all(['li', 'p'], class_='rtejustify')

            # Extraer el campo ocupacional
        campo_ocupacional_div = soup.find('div', id='campo-ocupacional')
        campo_ocupacional_items = campo_ocupacional_div.find_all('li')

        # Extraer el perfil profesional
        perfil_profesional_div = soup.find('div', id='perfil-info')
        perfil_profesional_items = perfil_profesional_div.find_all('li')

        if perfil_profesional_items:
            perfil_profesional = '\n'.join(item.get_text(strip=True) for item in perfil_profesional_items)
        else:
            perfil_profesional = "Perfil profesional no encontrado"

            
        if campo_ocupacional_items:
            campo_ocupacional = '\n'.join(item.get_text(strip=True) for item in campo_ocupacional_items)
        else:
            campo_ocupacional = "Campo ocupacional no encontrado"                 
            
        if descripcion_items:
            descripcion = ' '.join(item.text.strip() for item in descripcion_items)
        else:
            descripcion = "Descripción no encontrada"
                
        # Crea una fila por cada materia y ciclo para cada modalidad
        for materia in materias:
            for modalidad in modalidades:
                extracted_data = {
                    'carrera': carrera,
                    'ciclo': ciclo,
                    'materia': materia,
                    'facultad': facultad,
                    'universidad': universidad,
                    'modalidad': modalidad,
                    'descripcion': descripcion,
                    'titulo': titulo.split(":", 1)[1].strip(),  # Extraer el texto después de ":"
                    'duracion': duracion.split(":", 1)[1].strip(),  # Extraer el texto después de ":"
                    'campo_ocupacional': campo_ocupacional, 
                    'perfil_profesional': perfil_profesional, 
                    'tipo': 'grado' 
                }
                writer.writerow(extracted_data)
   


In [64]:


def scrape_and_save_data_posgrado(soup, writer):
    # Código para extraer datos de posgrado
    
    titulo_maestria = soup.find("h2").text.strip()
    titulo_maestria = "Maestría en " + titulo_maestria  # Agregar "Maestría en" al principio
    perfil_profesional_div = soup.find('div', id='tabs-1')
    perfil_profesional_items = perfil_profesional_div.find_all('li')
    if perfil_profesional_items:
        perfil_profesional = '\n'.join(item.get_text(strip=True) for item in perfil_profesional_items)
    else:
        perfil_profesional = "Perfil profesional no encontrado"

    titulo_img = soup.find('img', src='https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png')
    print(titulo_img)
    if titulo_img:
        titulo = titulo_img.find_next('p').text.strip()
    else:
        titulo = "Título no encontrado"
    modalidad_img = soup.find('img', src='https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/modalidad-ic.png')
    if modalidad_img:
        modalidad = modalidad_img.find_next('p').text.strip()
    else:
        modalidad = "Modalidad no encontrada"
    duracion_img = soup.find('img', src='https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/duracion-ic.png')
    if duracion_img:
        duracion = duracion_img.find_next('p').text.strip()
    else:
        duracion = "Duración no encontrada"
        
    extracted_data = {
        
        'perfil_profesional': perfil_profesional,
        'carrera': titulo_maestria,
        'titulo': titulo,
        'modalidad': modalidad,
        'duracion': duracion,
        'universidad': 'Universidad Técnica Particular de Loja',
        'tipo': 'posgrado'
    }
    writer.writerow(extracted_data)
    print(titulo_maestria)


In [96]:
def scrape_and_save_data_tecnologias(soup, writer):
    titulo_maestria = soup.find("div", class_='titulo').text.strip()
    descripcion_div = soup.find('div', class_='bloque1')
    descripcion = descripcion_div.find('p').text.strip()
    titulo_otorga_elem = descripcion_div.find(string='Título que otorga:')
    
    print(titulo_otorga_elem)


    # titulo = soup.find_next('p').text.strip()

    # print(titulo)


    print("Código para extraer datos de tecnologías")



In [17]:
def scrape_and_save_data(url, writer):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        if 'maestrias' in url:
            scrape_and_save_data_posgrado(soup, writer)
        elif 'tec.utpl.edu.ec' in url:
            scrape_and_save_data_tecnologias(soup, writer)
        else:
            scrape_and_save_data_grado(soup, writer)
    else:
        print("No se pudo acceder a la página web:", url)

In [68]:
def main():
    start_url = 'https://www.utpl.edu.ec'
    carrera_links = parse_carrera_links(start_url)
    
    csv_file = 'datos_universidades.csv'
    file_exists = os.path.exists(csv_file)
    with open(csv_file, 'a', newline='', encoding='utf-8') as file:
        fieldnames = ['carrera', 'ciclo', 'facultad', 'materia', 'universidad', 'modalidad', 'descripcion', 'titulo', 'duracion', 'campo_ocupacional', 'perfil_profesional', 'tipo']
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=';')
        
        if not file_exists:
            writer.writeheader()
        
        # scrape_and_save_data('https://www.utpl.edu.ec/maestrias/alimentos', writer)
        for carrera_link in carrera_links:
            full_url = start_url + carrera_link if carrera_link.startswith('/') else carrera_link
            scrape_and_save_data(full_url, writer)


    print("Datos guardados en el archivo CSV:", csv_file)

In [100]:
    
    
if __name__ == "__main__":
    main()

<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Alimentos
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Arquitectura con mención en Vivienda de Interés Social
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Ciencias y Tecnologías de la Computación
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Comunicación Estratégica mención Comunicación Digital
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Cooperación Internacional para el Desarrollo Sostenible
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Derecho Constitucional
<img src="https://www.utpl.edu.ec/maestrias/sites/all/themes/utpl/images/img/titulo-ic.png"/>
Maestría en Derecho mención Der